In [9]:
from prepross import *
import numpy as np
import os
from dataloader import Vocab
pd.options.display.max_rows = 100
from embedding import Doc2VecEmbedding
import pickle
import numpy as np

In [2]:
class Parse():
    def __init__(self):
        self.test_mode = True
arg = Parse()

In [3]:
config = Config(arg)

In [4]:
config.exp_dir

'/home/mehrazin/Documents/DAV_project/Repository/Music-Recommendation/Dumped/5'

In [5]:
df = Data_handler(config, pass_col = ['lyrics'])
# df.data = df.get_masked_data(users = df.users[:3], fracs = [0.1,0.2,0.1])
# df.data = create_session(df.data, config)
# df.data = clean_data(df.data, config)
# df.data = cut_sessions(df.data, config)
# config.clean_mode = ['rm_small_sub_session']
# df.data = clean_data(df.data, config)
# df.data

In [6]:
df.data.dropna(inplace = True)
df.data['lyrics'] = df.data['lyrics'].apply(lambda x: x.replace("\n", " "))
df.data.reset_index(inplace = True, drop = True)

In [7]:
df.data

,artist_name,track_name,lyrics
0,Artful Dodger,What You Gonna Do?,"Drop the funk, drop the bass Headed, headed fo..."
1,Herbie Hancock,The Essence,This is history right here in the makin' So ca...
2,Radiohead,Idioteque,Who's in a bunker? Who's in a bunker? Women an...
3,Black Jazz Chronicles,Dope Stuff,CDsAC/DC - Back in Black (1980) - For Those Ab...
4,Röyksopp,Only This Moment,Only this (moment) Holds us (together) Close t...
...,...,...,...
16651,Chiasm,Isolated,Isolated Alone and apart Isolated Isolated I'l...
16652,Vnv Nation,Tempest,"These are not words, they're only feelings The..."
16653,Apoptygma Berzerk,Coma White,There's something cold and blank behind her sm...
16654,Hocico,Poltergeist,Turn on your senses with the last moonlight te...


In [8]:
doc2vec = Doc2VecEmbedding(data = df.data, config = config)

  0%|          | 23/16656 [00:00<01:39, 166.96it/s]

tagging documents


100%|██████████| 16656/16656 [03:34<00:00, 77.50it/s] 


training doc2vec


  0%|          | 0/16656 [00:00<?, ?it/s]

inferring vectors


100%|██████████| 16656/16656 [29:01<00:00,  9.56it/s] 


In [23]:
path = os.path.join(config.exp_dir, 'df_doc2vec.csv')
# with open(path, 'rb') as f:
#     vec = pickle.load(f)
emb = pd.read_csv(path)

In [12]:
vec.shape

(16656, 500)

In [15]:
it = df.data.iterrows()
t , r = next(it)

In [20]:
df.data.reset_index(inplace = True, drop = True)
df.data.reset_index(inplace = True)

In [21]:
df.data

,index,artist_name,track_name,lyrics
0,0,Artful Dodger,What You Gonna Do?,"Drop the funk, drop the bass Headed, headed fo..."
1,1,Herbie Hancock,The Essence,This is history right here in the makin' So ca...
2,2,Radiohead,Idioteque,Who's in a bunker? Who's in a bunker? Women an...
3,3,Black Jazz Chronicles,Dope Stuff,CDsAC/DC - Back in Black (1980) - For Those Ab...
4,4,Röyksopp,Only This Moment,Only this (moment) Holds us (together) Close t...
...,...,...,...,...
16651,16651,Chiasm,Isolated,Isolated Alone and apart Isolated Isolated I'l...
16652,16652,Vnv Nation,Tempest,"These are not words, they're only feelings The..."
16653,16653,Apoptygma Berzerk,Coma White,There's something cold and blank behind her sm...
16654,16654,Hocico,Poltergeist,Turn on your senses with the last moonlight te...


In [29]:
dic = {(row['artist_name'], row['track_name']) : vec[int(row['index'])] for _,row in df.data.iterrows()}

In [31]:
name = 'item2emb.pkl'
path = os.path.join(config.exp_dir, name)
with open(path, 'wb') as f:
    pickle.dump(dic, f)

In [5]:
# df.data.to_csv(os.path.join(config.Dataset_dir, 'test_tab.tsv'), sep = '\t', index=False, header = False)